Superstore Sales Dataset için modellerimizi kaydedeceğiz

In [1]:
DATA_FILE = "train.csv"

In [2]:
MODEL_FILE = "superstore_models.pkl"

In [3]:
def train_and_save():
    print("🚀 PROFESYONEL EĞİTİM BAŞLIYOR (GRID SEARCH)...")
    
    if not os.path.exists(DATA_FILE):
        print(f"❌ HATA: '{DATA_FILE}' bulunamadı!")
        return

    df = pd.read_csv(DATA_FILE)
    df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
    df_monthly = df.groupby(['Category', pd.Grouper(key='Order Date', freq='ME')])['Sales'].sum().reset_index()
    
    categories = df_monthly['Category'].unique()
    best_models = {}
    
    # Grid Search Parametreleri
    trends = ['add', 'mul', None]
    seasonals = ['add', 'mul', None]
    dampeds = [True, False]
    use_logs = [True, False]
    
    for cat in categories:
        print(f"\n🏆 Kategori Yarıştırılıyor: {cat}")
        
        # Veriyi Hazırla
        cat_data = df_monthly[df_monthly['Category'] == cat].set_index('Order Date')['Sales']
        cat_data.index.freq = 'ME'
        
        # Son 12 ayı validasyon (test) için ayır
        train = cat_data.iloc[:-12]
        val = cat_data.iloc[-12:]
        
        best_score = float('inf')
        best_cfg = None
        best_model_obj = None
        
        # Kombinasyonları Dene
        for log in use_logs:
            for t in trends:
                for s in seasonals:
                    for d in dampeds:
                        # Uyumsuz parametreleri atla
                        if t is None and d: continue # Trend yoksa sönümleme olamaz
                        if s is None: continue # Mevsimsellik şart (verimiz mevsimsel)
                        
                        try:
                            # Veri Hazırlığı
                            if log:
                                train_loop = np.log1p(train)
                                val_loop = np.log1p(val)
                            else:
                                train_loop = train
                                val_loop = val
                            
                            # Eğit
                            model = ExponentialSmoothing(
                                train_loop,
                                seasonal_periods=12,
                                trend=t,
                                seasonal=s,
                                damped_trend=d
                            ).fit(optimized=True)
                            
                            # Tahmin
                            pred = model.forecast(12)
                            
                            # Skoru Hesapla (MAPE)
                            # Log ise geri çevirip hesapla ki adil olsun
                            if log:
                                pred_final = np.expm1(pred)
                                val_final = np.expm1(val_loop)
                            else:
                                pred_final = pred
                                val_final = val_loop
                                
                            mape = np.mean(np.abs((val_final - pred_final) / val_final)) * 100
                            
                            if mape < best_score:
                                best_score = mape
                                best_cfg = {'log': log, 'trend': t, 'seasonal': s, 'damped': d}
                                print(f"   ✨ Yeni Lider: MAPE %{mape:.2f} | Log: {log} | Trend: {t} | Season: {s}")
                                
                        except:
                            continue

        print(f"   ✅ KAZANAN: MAPE %{best_score:.2f} (Ayarlar: {best_cfg})")
        
        # ŞAMPİYON MODELİ TÜM VERİYLE YENİDEN EĞİT
        if best_cfg['log']:
            final_data = np.log1p(cat_data)
        else:
            final_data = cat_data
            
        final_model = ExponentialSmoothing(
            final_data,
            seasonal_periods=12,
            trend=best_cfg['trend'],
            seasonal=best_cfg['seasonal'],
            damped_trend=best_cfg['damped']
        ).fit(optimized=True)
        
        best_models[cat] = {'model': final_model, 'config': best_cfg}

    # Kaydet
    joblib.dump(best_models, MODEL_FILE)
    print(f"\n💾 Şampiyon modeller '{MODEL_FILE}' dosyasına kaydedildi.")

In [4]:
import pandas as pd
import numpy as np
import joblib
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import os

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
if __name__ == "__main__":
    train_and_save()

🚀 PROFESYONEL EĞİTİM BAŞLIYOR (GRID SEARCH)...

🏆 Kategori Yarıştırılıyor: Furniture
   ✨ Yeni Lider: MAPE %31.48 | Log: True | Trend: add | Season: add
   ✨ Yeni Lider: MAPE %29.27 | Log: True | Trend: None | Season: add
   ✨ Yeni Lider: MAPE %28.87 | Log: True | Trend: None | Season: mul
   ✅ KAZANAN: MAPE %28.87 (Ayarlar: {'log': True, 'trend': None, 'seasonal': 'mul', 'damped': False})

🏆 Kategori Yarıştırılıyor: Office Supplies
   ✨ Yeni Lider: MAPE %23.39 | Log: True | Trend: add | Season: add
   ✨ Yeni Lider: MAPE %23.12 | Log: True | Trend: add | Season: add
   ✅ KAZANAN: MAPE %23.12 (Ayarlar: {'log': True, 'trend': 'add', 'seasonal': 'add', 'damped': False})

🏆 Kategori Yarıştırılıyor: Technology
   ✨ Yeni Lider: MAPE %32.07 | Log: True | Trend: add | Season: add
   ✨ Yeni Lider: MAPE %32.01 | Log: True | Trend: add | Season: mul
   ✨ Yeni Lider: MAPE %31.14 | Log: True | Trend: None | Season: add
   ✅ KAZANAN: MAPE %31.14 (Ayarlar: {'log': True, 'trend': None, 'seasonal': 'ad